##  Imports

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential

In [3]:
from keras.layers import InputLayer, Conv2D, Dense, MaxPool2D, Dropout, Flatten, ZeroPadding2D, Lambda

In [4]:
from IPython.display import FileLink

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
from keras.initializers import RandomUniform

In [7]:
from keras.optimizers import Adam, RMSprop

In [8]:
from keras.activations import relu,softmax

In [9]:
from keras.models import load_model

## Aquesta funció la utilizo per fer grafics de percentatge d'encert 

In [10]:

import matplotlib
import numpy as np
import matplotlib.pyplot as plt


def plotHistory(history):
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
%matplotlib inline


In [14]:
from os.path import join

# ENTRADA I TRACTAMENT DE DADES

### Aquestes dues ce•les les utilitzo per utilitzar augmentacio de dades o no

In [14]:
idg = ImageDataGenerator()

In [12]:
idg = ImageDataGenerator( zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2)

## NIST DATASET

### Aquestes dues cel•les les utilitzo per triar entre el set de dades complert o un subset mes petit per fer proves

In [10]:
path = "./data/nist/"

In [13]:
path = "./data/nist/sample"

## MNIST DATASET

In [22]:
path = "./data/mnist/"

###  Carrega de dades i preprocessat d'imatges (escala de grisos i 64x64px)


In [15]:
batches = idg.flow_from_directory(join(path, "train"), target_size=(64,64), color_mode="grayscale")
val_batches = idg.flow_from_directory(join(path, "val"), target_size=(64,64), color_mode="grayscale")

Found 406212 images belonging to 26 classes.
Found 5090 images belonging to 26 classes.


### Definició del model 

In [20]:
model = Sequential()
model.add(InputLayer(input_shape=(64,64,1)))


model.add(Lambda(function=lambda x: 255 - x))
model.add(Conv2D(64,3,  data_format=batches.data_format, padding="same", activation="relu"))
model.add(Conv2D(64,3,  data_format=batches.data_format, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(128,3,  data_format=batches.data_format, padding="same", activation="relu"))
model.add(Conv2D(128,3,  data_format=batches.data_format, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))


model.add(Dense(26,activation="softmax"))
#model.add(Dense(10,activation="softmax"))




In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       1280      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16777344  
__________

In [15]:
#Aquesta cel•la la utilitzo quan vull carregar un model entrenat anteriorment
model = load_model("./checkpoint_nonums.h5")

In [21]:
#Aquesta cel•la la utilizto quan vull començar un model
model.compile(Adam(), "categorical_crossentropy", metrics=["accuracy"])

In [22]:
#Amb aquesta callback, sempre despres d'un epoch tinguem millors resultats que els anteriors, guarda el model en un fitxer
callback = keras.callbacks.ModelCheckpoint("./checkpoint_mnist_test.h5", monitor='val_loss', verbose=1, save_best_only=True,  mode='auto')

### Entrenament del model

In [23]:
history = model.fit_generator(batches, 1000, epochs=5, validation_data=val_batches, validation_steps=val_batches.samples/val_batches.batch_size, verbose=2, callbacks=[callback])

Epoch 1/5
Epoch 00000: val_loss improved from inf to 14.80467, saving model to ./checkpoint_mnist_test.h5
87s - loss: 14.9829 - acc: 0.0694 - val_loss: 14.8047 - val_acc: 0.0815
Epoch 2/5
Epoch 00001: val_loss improved from 14.80467 to 14.80467, saving model to ./checkpoint_mnist_test.h5
78s - loss: 14.8135 - acc: 0.0808 - val_loss: 14.8047 - val_acc: 0.0815
Epoch 3/5
Epoch 00002: val_loss improved from 14.80467 to 14.79631, saving model to ./checkpoint_mnist_test.h5
78s - loss: 14.7708 - acc: 0.0836 - val_loss: 14.7963 - val_acc: 0.0820
Epoch 4/5
Epoch 00003: val_loss improved from 14.79631 to 14.72573, saving model to ./checkpoint_mnist_test.h5
78s - loss: 14.7285 - acc: 0.0862 - val_loss: 14.7257 - val_acc: 0.0864
Epoch 5/5
Epoch 00004: val_loss did not improve
77s - loss: 14.7262 - acc: 0.0864 - val_loss: 14.8348 - val_acc: 0.0796
